In [1]:
import cv2
import numpy as np
import PIL
from PIL import Image

In [12]:
# 1. 特征提取
def extract_features(image):
    feature_extractor = cv2.SIFT_create(nfeatures=10000, nOctaveLayers=4, contrastThreshold=0.02, edgeThreshold=10, sigma=1.0)
    keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
    return keypoints, descriptors

# 2. 特征匹配
def match_features(descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

# 3. 变换估计
def estimate_transform(keypoints1, keypoints2, good_matches):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    # 使用RANSAC算法估计变换矩阵
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return M

# 4. 变换应用并进行图像拼接
# 4. 变换应用并进行图像拼接
def stitch_images(image1, image2):
    keypoints1, descriptors1 = extract_features(image1)
    keypoints2, descriptors2 = extract_features(image2)

    good_matches = match_features(descriptors1, descriptors2)
    M = estimate_transform(keypoints1, keypoints2, good_matches)

    h1, w1 = image1.shape
    h2, w2 = image2.shape
    new_height = max(h1, h2)
    new_width = w1 + w2

    # 创建一个空白的拼接图像
    stitched_image = np.zeros((new_height, new_width), dtype=np.uint8)

    # 应用变换到第一幅图像
    transformed_image = cv2.warpPerspective(image1, M, (new_width, new_height))

    # 将第一幅图像复制到拼接图像的左侧
    stitched_image[:, :w1] = transformed_image

    # 将第二幅图像复制到拼接图像的右侧
    stitched_image[:, w1:] = image2

    return stitched_image

In [13]:
# 读取多幅图像
image_paths = ['origin_images/2.png', 'origin_images/222.png']
stitched_image = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)

for i in range(1, len(image_paths)):
    next_image = cv2.imread(image_paths[i], cv2.IMREAD_GRAYSCALE)
    stitched_image = stitch_images(stitched_image, next_image)

# 保存拼接后的图像
cv2.imwrite('panorama_image.png', stitched_image)

ValueError: could not broadcast input array from shape (3036,8048) into shape (3036,4024)

In [21]:
image2 = cv2.imread('origin_images/2.png')
print(image2.shape)

(3036, 4024, 3)


In [26]:
import cv2
import numpy as np
# 读取两张图像
image1 = cv2.imread('origin_images/2.png')
image2 = cv2.imread('origin_images/442.png')
# 创建特征点检测器和描述子提取器
# 修改ORB特征提取函数
# 1. 特征提取
def extract_features(image):
    feature_extractor = cv2.SIFT_create(nfeatures=10000, nOctaveLayers=4, contrastThreshold=0.02, edgeThreshold=10, sigma=1.0)
    keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
    return keypoints, descriptors
def match_features(descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

# 对image1和image2分别应用ORB特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)
# 使用BFMatcher进行特征点匹配
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
good_matches = match_features(descriptors1, descriptors2)

# 提取匹配点对应的特征点坐标
points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
# 使用RANSAC算法估计两张图像之间的透视变换矩阵
M, _ = cv2.findHomography(points1, points2, cv2.RANSAC, 5.0)
# 对图像1进行透视变换，将其拼接到图像2上
result = cv2.warpPerspective(image1, M, (image1.shape[1] + image2.shape[1], image2.shape[0]))
result[0:image2.shape[0], 0:image2.shape[1]] = image2
# 显示拼接结果
# cv2.imshow('Image Stitching', result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite("panorama_image.png", result)

True

In [28]:

zero_pixel_count = result.size - cv2.countNonZero(result)
print("像素值为0的点的数量：", zero_pixel_count)
print("像素为0的点的占比 ：",f'{(zero_pixel_count/result.size) :.2%}')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\count_non_zero.dispatch.cpp:124: error: (-215:Assertion failed) cn == 1 in function 'cv::countNonZero'
